In [ ]:
"""
# PPMI SBR Calculation & Baseline Modeling

This notebook implements Striatal Binding Ratio (SBR) calculation and baseline machine learning models.
"""

In [ ]:
# Cell 1: Imports and Setup
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'src'))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                           roc_curve, precision_recall_curve, accuracy_score)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
print("Libraries imported successfully!")

In [ ]:
# Cell 2: Load Data and Prepare for SBR Calculation
from data.ppmi_custom_loader import load_ppmi_data
from features.sbr_calculator import SBRCalculator
from utils.config import get_config

In [ ]:
# Load PPMI data
print("Loading PPMI data...")
mapping_df, summary = load_ppmi_data()
print(f"Dataset loaded: {summary['total_images']} images from {summary['unique_patients']} patients")

In [ ]:
# Initialize SBR calculator
config = get_config()
sbr_calculator = SBRCalculator(config)

In [ ]:
# Select a subset for analysis (to avoid memory issues)
sample_size = min(50, len(mapping_df))
sample_mapping = mapping_df.sample(n=sample_size, random_state=42)
print(f"Selected {sample_size} images for SBR analysis")

In [ ]:
# Cell 3: SBR Feature Calculation
print("Calculating SBR features...")
try:
    sample_features = sbr_calculator.calculate_sbr_dataset(sample_mapping)
    print(f"✅ SBR features calculated successfully!")
    print(f"Feature shape: {sample_features.shape}")
    print(f"Feature columns: {list(sample_features.columns)}")
    
    # Show basic statistics
    print("\nFeature Summary:")
    print(sample_features.describe())

In [ ]:
except Exception as e:
    print(f"❌ SBR calculation failed: {e}")
    print("Creating mock features for demonstration...")
    
    # Create mock features for demonstration
    np.random.seed(42)
    mock_features = []
    
    for idx, row in sample_mapping.iterrows():
        mock_feature = {
            'patient_id': row['patient_id'],
            'series_path': row['file_path'],
            'label': np.random.choice([0, 1]),  # Mock labels
            'sbr_left_caudate': np.random.normal(2.5, 0.5),
            'sbr_right_caudate': np.random.normal(2.4, 0.5),
            'sbr_left_putamen': np.random.normal(2.3, 0.5),
            'sbr_right_putamen': np.random.normal(2.2, 0.5),
            'volume_mean': np.random.normal(1000, 200),
            'volume_std': np.random.normal(500, 100),
            'asymmetry_index': np.random.normal(0.05, 0.1),
            'entropy': np.random.normal(8.0, 1.0),
            'age': pd.to_numeric(row.get('age', 65), errors='coerce'),
            'sex_encoded': 1 if row.get('sex') == 'M' else 0
        }
        mock_features.append(mock_feature)
    
    sample_features = pd.DataFrame(mock_features)
    print(f"✅ Mock features created: {sample_features.shape}")

In [ ]:
# Cell 4: Feature Engineering and Preprocessing
print("\n🔧 Feature Engineering and Preprocessing...")

In [ ]:
# Prepare features for modeling
feature_cols = [col for col in sample_features.columns 
                if col not in ['series_path', 'patient_id', 'label']]

In [ ]:
X = sample_features[feature_cols].fillna(0)
y = sample_features['label']

In [ ]:
print(f"Features (X): {X.shape}")
print(f"Labels (y): {y.shape}")
print(f"Class distribution: {y.value_counts().to_dict()}")

In [ ]:
# Feature correlation analysis
plt.figure(figsize=(12, 8))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance ranking
from sklearn.ensemble import RandomForestClassifier
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selector.fit(X, y)

In [ ]:
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_selector.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importance)), feature_importance['importance'])
plt.title('Feature Importance Ranking')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(range(len(feature_importance)), feature_importance['feature'], rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

In [ ]:
# Cell 5: Data Splitting and Scaling
print("\n📊 Data Splitting and Scaling...")

In [ ]:
# Split data (stratified by patient to avoid data leakage)
patient_ids = sample_features['patient_id']
unique_patients = patient_ids.unique()

In [ ]:
# Create patient-level splits
patient_train, patient_test = train_test_split(
    unique_patients, test_size=0.3, random_state=42, 
    stratify=[y[patient_ids == pid].iloc[0] for pid in unique_patients]
)

In [ ]:
# Create image-level splits based on patient splits
train_mask = patient_ids.isin(patient_train)
test_mask = patient_ids.isin(patient_test)

In [ ]:
X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]

In [ ]:
print(f"Training set: {X_train.shape[0]} images from {len(patient_train)} patients")
print(f"Test set: {X_test.shape[0]} images from {len(patient_test)} patients")

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print("✅ Data splitting and scaling completed!")

In [ ]:
# Cell 6: Baseline Model Training
print("\n🎯 Training Baseline Models...")

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

In [ ]:
results = {}
trained_models = {}

In [ ]:
for name, model in models.items():
    print(f"\nTraining {name}...")
    
    try:
        # Train model
        model.fit(X_train_scaled, y_train)
        trained_models[name] = model
        
        # Make predictions
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        
        results[name] = {
            'accuracy': accuracy,
            'auc': auc,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }
        
        print(f"  ✅ {name} trained successfully!")
        print(f"  Accuracy: {accuracy:.3f}")
        print(f"  ROC AUC: {auc:.3f}")
        
    except Exception as e:
        print(f"  ❌ {name} training failed: {e}")

In [ ]:
print(f"\n✅ {len(trained_models)} models trained successfully!")

In [ ]:
# Cell 7: Model Performance Comparison
print("\n📈 Model Performance Comparison...")

In [ ]:
# Create performance comparison plot
if results:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Accuracy comparison
    model_names = list(results.keys())
    accuracies = [results[name]['accuracy'] for name in model_names]
    
    axes[0,0].bar(model_names, accuracies, alpha=0.7, color='skyblue')
    axes[0,0].set_title('Model Accuracy Comparison')
    axes[0,0].set_ylabel('Accuracy')
    axes[0,0].set_ylim(0, 1)
    for i, v in enumerate(accuracies):
        axes[0,0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # 2. ROC AUC comparison
    aucs = [results[name]['auc'] for name in model_names]
    
    axes[0,1].bar(model_names, aucs, alpha=0.7, color='lightgreen')
    axes[0,1].set_title('Model ROC AUC Comparison')
    axes[0,1].set_ylabel('ROC AUC')
    axes[0,1].set_ylim(0, 1)
    for i, v in enumerate(aucs):
        axes[0,1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # 3. ROC curves
    for name in model_names:
        y_pred_proba = results[name]['y_pred_proba']
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        auc = results[name]['auc']
        axes[1,0].plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})')
    
    axes[1,0].plot([0, 1], [0, 1], 'k--', label='Random')
    axes[1,0].set_xlabel('False Positive Rate')
    axes[1,0].set_ylabel('True Positive Rate')
    axes[1,0].set_title('ROC Curves Comparison')
    axes[1,0].legend()
    axes[1,0].grid(True)
    
    # 4. Precision-Recall curves
    for name in model_names:
        y_pred_proba = results[name]['y_pred_proba']
        precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
        axes[1,1].plot(recall, precision, label=f'{name}')
    
    axes[1,1].set_xlabel('Recall')
    axes[1,1].set_ylabel('Precision')
    axes[1,1].set_title('Precision-Recall Curves')
    axes[1,1].legend()
    axes[1,1].grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Cell 8: Detailed Model Analysis
print("\n🔍 Detailed Model Analysis...")

In [ ]:
if results:
    # Find best model
    best_model_name = max(results.keys(), key=lambda x: results[x]['auc'])
    best_model = trained_models[best_model_name]
    best_results = results[best_model_name]
    
    print(f"🏆 Best Model: {best_model_name}")
    print(f"   Accuracy: {best_results['accuracy']:.3f}")
    print(f"   ROC AUC: {best_results['auc']:.3f}")
    
    # Detailed classification report
    print(f"\n📋 Classification Report for {best_model_name}:")
    print(classification_report(y_test, best_results['y_pred']))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, best_results['y_pred'])
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Control', 'PD'], 
                yticklabels=['Control', 'PD'])
    plt.title(f'Confusion Matrix - {best_model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Feature importance for best model (if applicable)
    if hasattr(best_model, 'feature_importances_'):
        print(f"\n🔝 Feature Importance for {best_model_name}:")
        feature_importance_best = pd.DataFrame({
            'feature': feature_cols,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(feature_importance_best.head(10))
        
        # Plot feature importance
        plt.figure(figsize=(10, 6))
        plt.bar(range(len(feature_importance_best)), feature_importance_best['importance'])
        plt.title(f'Feature Importance - {best_model_name}')
        plt.xlabel('Features')
        plt.ylabel('Importance')
        plt.xticks(range(len(feature_importance_best)), 
                   feature_importance_best['feature'], rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

In [ ]:
# Cell 9: Cross-Validation Analysis
print("\n🔄 Cross-Validation Analysis...")

In [ ]:
if trained_models:
    # Perform cross-validation on the best model
    best_model_name = max(results.keys(), key=lambda x: results[x]['auc'])
    best_model = trained_models[best_model_name]
    
    # Patient-level cross-validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    
    # Group by patient for CV
    patient_labels = []
    patient_features = []
    
    for patient_id in unique_patients:
        patient_mask = patient_ids == patient_id
        if patient_mask.sum() > 0:
            patient_features.append(X[patient_mask].mean(axis=0))
            patient_labels.append(y[patient_mask].iloc[0])
    
    patient_features = np.array(patient_features)
    patient_labels = np.array(patient_labels)
    
    # Cross-validation scores
    cv_scores = cross_val_score(best_model, patient_features, patient_labels, 
                               cv=cv, scoring='roc_auc')
    
    print(f"Cross-validation ROC AUC scores: {cv_scores}")
    print(f"Mean CV ROC AUC: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

In [ ]:
# Cell 10: Summary and Next Steps
print("\n🎉 SBR Baseline Modeling Complete! 🎉")
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

In [ ]:
if results:
    print(f"📊 Models Trained: {len(trained_models)}")
    print(f"🏆 Best Model: {best_model_name}")
    print(f"🎯 Best ROC AUC: {max(results.values(), key=lambda x: x['auc'])['auc']:.3f}")
    print(f"📈 Best Accuracy: {max(results.values(), key=lambda x: x['accuracy'])['accuracy']:.3f}")

In [ ]:
print(f"\n🔬 Analysis Completed:")
print(f"  SBR feature calculation")
print(f"  Feature engineering and preprocessing")
print(f"  Multiple baseline models")
print(f"  Performance comparison")
print(f"  Cross-validation analysis")

In [ ]:
print("\n🚀 Next Steps:")
print("1. Implement proper SBR calculation with real labels")
print("2. Add more sophisticated features (texture, shape)")
print("3. Develop 3D CNN models")
print("4. Implement ensemble methods")
print("5. Add clinical feature integration")

In [ ]:
# Save results
if results:
    results_df = pd.DataFrame({
        'Model': list(results.keys()),
        'Accuracy': [results[name]['accuracy'] for name in results.keys()],
        'ROC_AUC': [results[name]['auc'] for name in results.keys()]
    }).sort_values('ROC_AUC', ascending=False)
    
    print(f"\n📊 Final Results Summary:")
    print(results_df)
    
    # Save to file
    output_dir = Path("results/reports")
    output_dir.mkdir(parents=True, exist_ok=True)
    results_df.to_csv(output_dir / "baseline_model_results.csv", index=False)
    print(f"\n💾 Results saved to {output_dir / 'baseline_model_results.csv'}")